# Titanicデータセットを用いた機械学習入門

<div style="display:flex;justify-content:center;">
    <img src="./assets/RMS_Titanic_3.jpg"  width="600px" alt="https://commons.wikimedia.org/wiki/File:RMS_Titanic_3.jpg" />
</div>

今回は、データサイエンスや機械学習を学ぶものの多くが通る道である、Titanicデータセットを用いたデータ分析を行っていきます。

もちろんタイタニックは、映画「タイタニック」で有名なあの船。

> タイタニック（英語: RMS Titanic、ロイヤルメールシップ・タイタニック）は、20世紀初頭に建造されたイギリス船籍のオーシャン・ライナー。
> ホワイト・スター・ライン社が保有するオリンピック級客船の2番船であったが、処女航海中の1912年4月14日深夜に氷山に衝突し、その際の損傷による浸水が原因となって翌15日未明に沈没した。([wikipediaより](https://ja.wikipedia.org/wiki/%E3%82%BF%E3%82%A4%E3%82%BF%E3%83%8B%E3%83%83%E3%82%AF_(%E5%AE%A2%E8%88%B9)))

## Titanic dataset について

Titanic datasetは色々なところでフリーのデータセットとして公開されています。
- Kaggle (https://www.kaggle.com/competitions/titanic)
- Tensorflow Datasets (https://www.tensorflow.org/datasets/catalog/titanic?hl=en)
- Seaborn Datasets (https://github.com/mwaskom/seaborn-data)
などなど

今回は、このチュートリアルに適して扱いやすい、Seaborn Datasetsのものを利用します。

## 機械学習プロジェクトの一般的な流れ

1. 分析の目的と問題設定
2. データを取得する
3. EDA ~ データからインサイトを得る ~
4. 前処理 ~ データクレンジングとフィーチャーエンジニアリング ~
5. モデルの作成、学習、推論の実行
<br>↑今回はここまで
6. モデルのファインチューニング
7. 結果の提示
8. システムに組み込む

## データを取得する

早速データを取得してみます。

In [1]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


## EDA ~ データからインサイトを得る ~

## 前処理 ~ データクレンジングとフィーチャーエンジニアリング ~

## モデルの作成、学習、推論の実行